In [1]:
import numpy as np
import pandas as pd

## Load Dataset

In [2]:
train = pd.read_csv("train.csv", index_col=["PassengerId"])

print(train.shape)
train.head()

(891, 11)


,Survived,Pclass,Name,Gender,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test = pd.read_csv("test.csv", index_col=["PassengerId"])

print(test.shape)
test.head()

(418, 10)


,Pclass,Name,Gender,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


## Preprocessing

### Merge train dataset and test dataset

In [4]:
combi = pd.concat([train, test])

print(combi.shape)
combi.head()

(1309, 11)


,Age,Cabin,Embarked,Fare,Gender,Name,Parch,Pclass,SibSp,Survived,Ticket
PassengerId,,,,,,,,,,,
1,22.0,NaN,S,7.2500,male,"Braund, Mr. Owen Harris",0,3,1,0.0,A/5 21171
2,38.0,C85,C,71.2833,female,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,1,1,1.0,PC 17599
3,26.0,NaN,S,7.9250,female,"Heikkinen, Miss. Laina",0,3,0,1.0,STON/O2. 3101282
4,35.0,C123,S,53.1000,female,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,1,1,1.0,113803
5,35.0,NaN,S,8.0500,male,"Allen, Mr. William Henry",0,3,0,0.0,373450


### Encode Gender

In [5]:
combi["Gender_encode"] = (combi["Gender"] == "male").astype(int)

print(combi.shape)
combi[["Gender", "Gender_encode"]].head()

(1309, 12)


,Gender,Gender_encode
PassengerId,,
1,male,1
2,female,0
3,female,0
4,female,0
5,male,1


### Fill out missing fare

In [6]:
mean_fare = train["Fare"].mean()

print("Fare(Mean) = ${0:.3f}".format(mean_fare))

Fare(Mean) = $32.204


In [7]:
combi["Fare_fillout"] = combi["Fare"]
combi.loc[pd.isnull(combi["Fare"]), "Fare_fillout"] = mean_fare

missing_fare = combi[pd.isnull(combi["Fare"])]

print(missing_fare.shape)
missing_fare[["Fare", "Fare_fillout"]].head()

(1, 13)


,Fare,Fare_fillout
PassengerId,,
1044,NaN,32.204208


### Encode Embarked

In [8]:
embarked = pd.get_dummies(combi["Embarked"], prefix="Embarked").astype(np.bool)

print(embarked.shape)
embarked.head()

(1309, 3)


,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,
1,False,False,True
2,True,False,False
3,False,False,True
4,False,False,True
5,False,False,True


In [9]:
combi = pd.concat([combi, embarked], axis=1)

print(combi.shape)
combi[["Embarked", "Embarked_C", "Embarked_Q", "Embarked_S"]].head()

(1309, 16)


,Embarked,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,
1,S,False,False,True
2,C,True,False,False
3,S,False,False,True
4,S,False,False,True
5,S,False,False,True


### Add Family

In [10]:
combi["Family"] = combi["SibSp"] + combi["Parch"]

print(combi.shape)
combi.head()

(1309, 17)


,Age,Cabin,Embarked,Fare,Gender,Name,Parch,Pclass,SibSp,Survived,Ticket,Gender_encode,Fare_fillout,Embarked_C,Embarked_Q,Embarked_S,Family
PassengerId,,,,,,,,,,,,,,,,,
1,22.0,NaN,S,7.2500,male,"Braund, Mr. Owen Harris",0,3,1,0.0,A/5 21171,1,7.2500,False,False,True,1
2,38.0,C85,C,71.2833,female,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,1,1,1.0,PC 17599,0,71.2833,True,False,False,1
3,26.0,NaN,S,7.9250,female,"Heikkinen, Miss. Laina",0,3,0,1.0,STON/O2. 3101282,0,7.9250,False,False,True,0
4,35.0,C123,S,53.1000,female,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,1,1,1.0,113803,0,53.1000,False,False,True,1
5,35.0,NaN,S,8.0500,male,"Allen, Mr. William Henry",0,3,0,0.0,373450,1,8.0500,False,False,True,0


### Split dataset into train and test

In [11]:
train = combi[pd.notnull(combi["Survived"])]

print(train.shape)
train.head()

(891, 17)


,Age,Cabin,Embarked,Fare,Gender,Name,Parch,Pclass,SibSp,Survived,Ticket,Gender_encode,Fare_fillout,Embarked_C,Embarked_Q,Embarked_S,Family
PassengerId,,,,,,,,,,,,,,,,,
1,22.0,NaN,S,7.2500,male,"Braund, Mr. Owen Harris",0,3,1,0.0,A/5 21171,1,7.2500,False,False,True,1
2,38.0,C85,C,71.2833,female,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,1,1,1.0,PC 17599,0,71.2833,True,False,False,1
3,26.0,NaN,S,7.9250,female,"Heikkinen, Miss. Laina",0,3,0,1.0,STON/O2. 3101282,0,7.9250,False,False,True,0
4,35.0,C123,S,53.1000,female,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,1,1,1.0,113803,0,53.1000,False,False,True,1
5,35.0,NaN,S,8.0500,male,"Allen, Mr. William Henry",0,3,0,0.0,373450,1,8.0500,False,False,True,0


In [12]:
test = combi[pd.isnull(combi["Survived"])]
test.drop("Survived", axis=1, inplace=True)

print(test.shape)
test.head()

(418, 16)


/Users/Rachel/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,Age,Cabin,Embarked,Fare,Gender,Name,Parch,Pclass,SibSp,Ticket,Gender_encode,Fare_fillout,Embarked_C,Embarked_Q,Embarked_S,Family
PassengerId,,,,,,,,,,,,,,,,
892,34.5,NaN,Q,7.8292,male,"Kelly, Mr. James",0,3,0,330911,1,7.8292,False,True,False,0
893,47.0,NaN,S,7.0000,female,"Wilkes, Mrs. James (Ellen Needs)",0,3,1,363272,0,7.0000,False,False,True,1
894,62.0,NaN,Q,9.6875,male,"Myles, Mr. Thomas Francis",0,2,0,240276,1,9.6875,False,True,False,0
895,27.0,NaN,S,8.6625,male,"Wirz, Mr. Albert",0,3,0,315154,1,8.6625,False,False,True,0
896,22.0,NaN,S,12.2875,female,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,3,1,3101298,0,12.2875,False,False,True,2


## Train

In [15]:
feature_names = ["Pclass", "Gender_encode", "Fare_fillout"]
feature_names = feature_names + list(embarked.columns)

label_name = "Survived"

In [16]:
X_train = train[feature_names]

print(X_train.shape)
X_train.head()

(891, 6)


,Pclass,Gender_encode,Fare_fillout,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,
1,3,1,7.2500,False,False,True
2,1,0,71.2833,True,False,False
3,3,0,7.9250,False,False,True
4,1,0,53.1000,False,False,True
5,3,1,8.0500,False,False,True


In [17]:
y_train = train[label_name]

print(y_train.shape)
y_train.head()

(891,)


PassengerId
1    0.0
2    1.0
3    1.0
4    1.0
5    0.0
Name: Survived, dtype: float64

In [18]:
seed = 37
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth=5,
                               random_state=seed)

## Score

In [20]:
X_test = test[feature_names]

print(X_test.shape)
X_test.head()

(418, 6)


,Pclass,Gender_encode,Fare_fillout,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,
892,3,1,7.8292,False,True,False
893,3,0,7.0000,False,False,True
894,2,1,9.6875,False,True,False
895,3,1,8.6625,False,False,True
896,3,0,12.2875,False,False,True


In [21]:
model.fit(X_train, y_train)
prediction = model.predict(X_test)

print(prediction.shape)
prediction[:20]

(418,)


array([ 0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  1.,
        0.,  1.,  1.,  0.,  0.,  0.,  1.])

## Submit

In [22]:
submission = pd.read_csv("gender_submission.csv", index_col="PassengerId")
submission["Survived"] = prediction.astype(np.int32)

print(submission.shape)
submission.head()

(418, 1)


,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,1


In [23]:
submission.to_csv("decision-tree-pclass-gender-fare-max-depth-5.csv")